**title: "Bulk rnaseq_mazzolini_foranna (Analysis for my PhD supervisor)"
author: "Lai, Kei Onn
date: "12/30/2022"**

dataset derived from count data of https://doi.org/10.1002/glia.23717
provided by original authors of the paper. 
Here i mainly focus on performing an LRT instead of wald statistic test since it's a time series model

read in the file; tab contains both metadata and count matrix

In [1]:
setwd("C:/Users/Kei Onn/Downloads")

.libPaths("C:/Program Files/R/R-4.2.1/library")


In [2]:
tab<-read.csv("count_bulk_ZF_Mazzolini.csv")
head(tab)

,sample,gene_id,count,gene_name,sample_group
,<chr>,<chr>,<int>,<chr>,<chr>
1,uz_hras_3dpf_16_5_17,ENSDARG00000000001,10,slc35a5,3dpf
2,uz_hras_3dpf_16_5_17,ENSDARG00000000002,2964,ccdc80,3dpf
3,uz_hras_3dpf_16_5_17,ENSDARG00000000018,194,nrf1,3dpf
4,uz_hras_3dpf_16_5_17,ENSDARG00000000019,2191,ube2h,3dpf
5,uz_hras_3dpf_16_5_17,ENSDARG00000000068,68,slc9a3r1a,3dpf
6,uz_hras_3dpf_16_5_17,ENSDARG00000000069,548,dap,3dpf


Extract out the metadata and gene annotations

In [3]:
metadata<-tab[,c("sample","sample_group")]
head(metadata)

,sample,sample_group
,<chr>,<chr>
1,uz_hras_3dpf_16_5_17,3dpf
2,uz_hras_3dpf_16_5_17,3dpf
3,uz_hras_3dpf_16_5_17,3dpf
4,uz_hras_3dpf_16_5_17,3dpf
5,uz_hras_3dpf_16_5_17,3dpf
6,uz_hras_3dpf_16_5_17,3dpf


Extract out the count matrix (originally, this is in the long format)

In [4]:
countlong<-tab[c("sample","gene_id","count")]
head(countlong)

,sample,gene_id,count
,<chr>,<chr>,<int>
1,uz_hras_3dpf_16_5_17,ENSDARG00000000001,10
2,uz_hras_3dpf_16_5_17,ENSDARG00000000002,2964
3,uz_hras_3dpf_16_5_17,ENSDARG00000000018,194
4,uz_hras_3dpf_16_5_17,ENSDARG00000000019,2191
5,uz_hras_3dpf_16_5_17,ENSDARG00000000068,68
6,uz_hras_3dpf_16_5_17,ENSDARG00000000069,548


In [5]:
library(stats)
countmatrix<-reshape(data=countlong,idvar="gene_id",timevar = "sample",direction = "wide")
rownames(countmatrix)=countmatrix$gene_id
countmatrix=subset(countmatrix, select=-c(gene_id))
head(countmatrix)

,count.uz_hras_3dpf_16_5_17,count.uz_hras_3dpf_9_3_17,count.uz_hras_3dpf_9_5_17,count.uz_hras_5dpf_11_5_17,count.uz_hras_5dpf_4_5_17_1,count.uz_hras_5dpf_4_5_17_2,count.uz_hras_7dpf_13_4_17,count.uz_hras_7dpf_20_4_17,count.uz_hras_7dpf_27_4_17
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
ENSDARG00000000001,10,4,20,38,64,2,1082,342,10
ENSDARG00000000002,2964,2845,411,672,347,1532,176,70,12
ENSDARG00000000018,194,251,181,673,686,405,576,273,378
ENSDARG00000000019,2191,1657,2574,1223,2208,2165,2929,884,4223
ENSDARG00000000068,68,303,2598,247,258,975,194,48,880
ENSDARG00000000069,548,1928,3849,68,41,290,59,504,57


bit of data wrangling to clean the colnames 

In [6]:

names(countmatrix) = gsub(pattern = "count.", replacement = "", x = names(countmatrix))



In [7]:
# for a sanity check
head(countmatrix)

,uz_hras_3dpf_16_5_17,uz_hras_3dpf_9_3_17,uz_hras_3dpf_9_5_17,uz_hras_5dpf_11_5_17,uz_hras_5dpf_4_5_17_1,uz_hras_5dpf_4_5_17_2,uz_hras_7dpf_13_4_17,uz_hras_7dpf_20_4_17,uz_hras_7dpf_27_4_17
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
ENSDARG00000000001,10,4,20,38,64,2,1082,342,10
ENSDARG00000000002,2964,2845,411,672,347,1532,176,70,12
ENSDARG00000000018,194,251,181,673,686,405,576,273,378
ENSDARG00000000019,2191,1657,2574,1223,2208,2165,2929,884,4223
ENSDARG00000000068,68,303,2598,247,258,975,194,48,880
ENSDARG00000000069,548,1928,3849,68,41,290,59,504,57


metadata has alot of replications in rows

In [8]:
dim(metadata)
head(metadata)

[1] 288513      2

,sample,sample_group
,<chr>,<chr>
1,uz_hras_3dpf_16_5_17,3dpf
2,uz_hras_3dpf_16_5_17,3dpf
3,uz_hras_3dpf_16_5_17,3dpf
4,uz_hras_3dpf_16_5_17,3dpf
5,uz_hras_3dpf_16_5_17,3dpf
6,uz_hras_3dpf_16_5_17,3dpf


In [9]:
suppressPackageStartupMessages(library(dplyr))
metadata<-metadata %>% distinct()
metadata <- metadata[-which(metadata$sample_group == ""), ]  #removing those samples not categorised as a sample group

metadata$sample_group<-factor(metadata$sample_group)

head(metadata)

Warning message:
"package 'dplyr' was built under R version 4.2.2"


,sample,sample_group
,<chr>,<fct>
1,uz_hras_3dpf_16_5_17,3dpf
3,uz_hras_3dpf_9_3_17,3dpf
5,uz_hras_3dpf_9_5_17,3dpf
7,uz_hras_5dpf_11_5_17,5pdf
9,uz_hras_5dpf_4_5_17_1,5pdf
11,uz_hras_5dpf_4_5_17_2,5pdf


time to create the summarized expt object.

first ensure that countmatrix same order as metadata

In [10]:
suppressPackageStartupMessages(library(DESeq2))

Warning message:
"package 'matrixStats' was built under R version 4.2.2"


In [11]:
all(metadata$sample == colnames(countmatrix))

[1] TRUE

In [12]:
#merge countmatrix with 

In [13]:
dds <- DESeqDataSetFromMatrix(countData = countmatrix,
                              colData = metadata,
                              design = ~ sample_group)

sample group has 3 replicate.
we filter at least total 10 reads across 3 samples

In [14]:
keep <- rowSums(counts(dds) >= 10) >= 3
dds <- dds[keep,]

3dpf is my reference group for contrast

In [15]:
dds$sample_group <- relevel(dds$sample_group, ref = "3dpf")

To get the genes that change at any time point 

In [ ]:
design(dds) <- ~sample_group
dds <- DESeq(dds, test="LRT", reduced=~1)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship



In [ ]:
LRTres <- results(dds, alpha = 0.05)
resultsNames(dds)
head(LRTres)

library(AnnotationHub)
library(ensembldb)
#Connect to AnnotationHub
ah <- AnnotationHub()

#Access the Ensembl database for organism
ahDb <- query(ah, 
              pattern = c("Danio rerio", "EnsDb"), 
              ignore.case = TRUE)



#Acquire the latest annotation files
id <- ahDb %>%
        mcols() %>%
        rownames() %>%
        tail(n = 1)

#Download the appropriate Ensembldb database
edb <- ah[[id]]

#Extract gene-level information from database
annotations <- genes(edb, 
                     return.type = "data.frame")

#Select annotations of interest
annotations <- annotations %>%
        dplyr::select(gene_id, gene_name, seq_name, gene_biotype, description)
saveRDS(annotations,file="ZFannotaions_ensdb.rds")

creating a gene list containg ensbl id and gene name

#this is the ZF annotation extracted from Annoation hub
#ZFannotaions_ensdb <- readRDS("C:/Users/Kei Onn/Downloads/ZFannotaions_ensdb.rds")

we extract the genes name for genes in count matrix

In [ ]:
annotations<-readRDS("C:/Users/Kei Onn/Downloads/ZFannotaions_ensdb.rds")
head(annotations)
dim(annotations)

In [ ]:
annotations<- annotations[annotations$gene_id%in%rownames(rowData(dds)),]
dim(annotations)

In [ ]:
mcols(dds) <- DataFrame(mcols(dds), annotations) #added annotations to my dds obj

In [ ]:
head(rowData(dds))

investigating patterns across time groups
aka the sample_groups

vst transformation on count matrix
next subsetting the vst transformed count matrix to only include
sig from the LRT

In [ ]:
suppressPackageStartupMessages(library(DEGreport))

In [ ]:
head(LRTres)

In [ ]:
vst_LRTdds<-vst(dds, blind=T)
head(vst_LRTdds)

we need to filter for significant genes before performing degPatterns()

In [ ]:
df_LRTres<-data.frame(LRTres@listData) %>% filter(padj<0.05) 
ma<-vst_LRTdds[rownames(assay(vst_LRTdds))%in%rownames(df_LRTres),]

In [ ]:
ma<-assay(ma)
clusterpatternplot<-degPatterns(
    ma = ma,
    metadata = colData(dds),
    time = "sample_group",
    minc = 50)#performs pairwise correlation between sig genes derived from LRT test and group them into clusters
    

In [ ]:
clusterpatternplot 

In [ ]:
DEGclusters<-clusterpatternplot[["df"]] # extracting the genes which belong to each cluster
head(DEGclusters)

We see that cluster 1 and 2 are age-dependant changes in the mg
that are linear; therefore we can select them here 

In [ ]:
cluster1<-subset(DEGclusters, cluster == 1, select = "genes")
cluster2<-subset(DEGclusters, cluster == 2, select = "genes")